# 1. Instalar librerias

In [1]:
!pip install pymongo

# 2. Cargar librerias

In [13]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
import chardet
import csv

# 3. Funciones

In [70]:
def DetectaEncoding(url):
  #Detectar el encoding de csv
  with open(url, 'rb') as f:
    result = chardet.detect(f.read(50000))
    encoding = result['encoding']
    if encoding.lower() == 'ascii':
      #Forzamos un encoding más permisivo
      encoding = 'latin-1'
    if encoding.lower() == 'big5':
      #Forzamos un encoding más permisivo
      encoding = 'cp1252'
  return encoding


def DetectaSeparador(url, num_lineas=5):
    with open(url, 'r', encoding=DetectaEncoding(url), errors='ignore') as f:
        muestra = ''.join([f.readline() for _ in range(num_lineas)])
        dialecto = csv.Sniffer().sniff(muestra)
        return dialecto.delimiter

def convierteFecha(df,columnas_fecha):
  for col in columnas_fecha:
    # df[col] = pd.to_datetime(df[col], format="%d-%m-%Y %H:%M:%S")
    df[col] = pd.to_datetime(df[col])
  return df


def mongoInsertMany(df,coleccion_nombre):
    try:
        #Asignar nombre a colección
        coleccion = bd[coleccion_nombre]

        #Reemplazar NaN por None (MongoDB no permite NaN)
        df_mongo = df.replace({np.nan: None})

        #transformamos a diccionario el df
        datos = df_mongo.to_dict(orient='records')

        #Insertamos los datos
        coleccion.insert_many(datos)
        print('Datos ingresados correctamente en ', bd , '---> ', coleccion_nombre)
    except Exception  as e:
        print("Ocurrió un error al insertar en MongoDB:", e)


# 4.Conexión

In [6]:
cliente = MongoClient("mongodb://localhost:27017/")  # Cambia si tu servidor es remoto
bd = cliente["hospital_db"]                          # Nombre de la base de datos
#coleccion = bd["ingresos"]    

# 5. Inserción

## 1. Ingresos

In [15]:
url = "Limpios/01_ingresos_limpio.csv";
df_ingresos =  pd.read_csv(url,delimiter=DetectaSeparador(url),encoding=DetectaEncoding(url))
df_ingresos

,ID_PACIENTE,EDAD,SEXO,DIAGNOSTICO_INGRESO,FECHA_INGRESO,FECHA_UCI_INGRESO,FECHA_UCI_SALIDA,UCI_DIAS,FECHA_ALTA_INGRESO,MOTIVO_ALTA_INGRESO,...,GLU_ULTIMA_URGENCIA,SAT_02_ULTIMA_URGENCIA,TA_MAX_ULTIMA_URGENCIA,TA_MIN_ULTIMA_URGENCIA,FUENTE,DIURESIS_PRIMERA_URGENCIA,FECHA_INGRESO_ANTERIOR,DIAGNOSTICO_ANTERIOR,RESPIRADOR,UCI_N_ING
0,577,78,MALE,COVID POSITIVO,2019-12-26,2019-12-26 17:12:00,2019-12-27 12:22:00,1.0,2020-04-02,DOMICILIO,...,NaN,NaN,NaN,NaN,F_24_04_2020,NaN,NaN,NaN,False,NaN
1,44,75,FEMALE,COVID POSITIVO,2020-01-28,2020-01-30 13:03:00,2020-01-31 17:08:00,1.0,2020-04-04,DOMICILIO,...,NaN,NaN,NaN,NaN,F_24_04_2020,NaN,NaN,NaN,False,NaN
2,585,62,FEMALE,COVID POSITIVO,2020-02-05,2020-03-10 14:20:00,2020-03-20 14:11:00,10.0,2020-03-26,DOMICILIO,...,NaN,97.0,160.0,101.0,F_24_04_2020,NaN,NaN,NaN,False,NaN
3,587,69,MALE,COVID POSITIVO,2020-02-06,NaN,NaN,NaN,2020-03-24,DOMICILIO,...,NaN,95.0,141.0,83.0,F_24_04_2020,NaN,NaN,NaN,False,NaN
4,586,67,FEMALE,COVID POSITIVO,2020-02-06,NaN,NaN,NaN,2020-03-28,FALLECIMIENTO,...,NaN,93.0,151.0,82.0,F_24_04_2020,NaN,NaN,NaN,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9328,2742,61,MALE,COVID POSITIVO,2020-03-22,NaN,NaN,NaN,2020-03-24,TRASLADO AL HOSPITAL,...,NaN,88.0,130.0,88.0,F_19_04_2021,0.0,2020-03-24,COVID CONFIRMADO,True,NaN
9329,2743,54,MALE,COVID POSITIVO,2020-03-22,NaN,NaN,NaN,2020-04-02,DOMICILIO,...,NaN,87.0,108.0,73.0,F_19_04_2021,0.0,NaN,NaN,True,NaN
9330,2744,77,MALE,COVID POSITIVO,2020-03-23,NaN,NaN,NaN,2020-04-07,DOMICILIO,...,NaN,98.0,NaN,NaN,F_19_04_2021,0.0,2020-04-10,NEUTROPENIA FEBRIL,True,NaN
9331,2745,71,MALE,COVID POSITIVO,2020-03-22,2020-03-27 19:06:27,2020-04-27 13:32:57,31.0,2020-05-22,DOMICILIO,...,NaN,97.0,110.0,72.0,F_19_04_2021,0.0,2020-07-09,AIT,True,4.0


In [ ]:
#CONVERTIR COLUMNAS EN FECHAS PARA QUE MONGO LAS DETECTE COMO TIPO DATE
# FECHA_INGRESO         
# FECHA_UCI_INGRESO     
# FECHA_UCI_SALIDA      
# FECHA_ALTA_INGRESO    
# FECHA_URGENCIA_INGRESO
# FECHA_INGRESO_ANTERIOR

columnas_fechas = ['FECHA_INGRESO', 'FECHA_UCI_INGRESO','FECHA_UCI_SALIDA','FECHA_ALTA_INGRESO','FECHA_URGENCIA_INGRESO','FECHA_INGRESO_ANTERIOR']
df_ingresos =  convierteFecha(df_ingresos, columnas_fechas)

In [29]:
df_ingresos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9333 entries, 0 to 9332
Data columns (total 34 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   ID_PACIENTE                       9333 non-null   int64         
 1   EDAD                              9333 non-null   int64         
 2   SEXO                              9333 non-null   object        
 3   DIAGNOSTICO_INGRESO               9333 non-null   object        
 4   FECHA_INGRESO                     9333 non-null   datetime64[ns]
 5   FECHA_UCI_INGRESO                 845 non-null    datetime64[ns]
 6   FECHA_UCI_SALIDA                  827 non-null    datetime64[ns]
 7   UCI_DIAS                          3146 non-null   float64       
 8   FECHA_ALTA_INGRESO                9112 non-null   datetime64[ns]
 9   MOTIVO_ALTA_INGRESO               8866 non-null   object        
 10  FECHA_URGENCIA_INGRESO            8968 non-null 

In [ ]:
#Insertamos ingresos en MongoDB
mongoInsertMany(df_ingresos,"ingresos")

Datos ingresados correctamente en  Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'hospital_db') ---> ingresos


## 2. Medicación

In [43]:
url = "Limpios/02_medicacion_limpio.csv";
df_medicacion =  pd.read_csv(url,delimiter=DetectaSeparador(url),encoding=DetectaEncoding(url))
df_medicacion

,ID_PACIENTE,FARMACO_NOMBRE_COMERCIAL,DOSIS_MEDIA_DIARIA,INICIO_TRATAMIENTO,FIN_TRATAMIENTO,ATC5_NOMBRE,ID_ATC5,ATC7_NOMBRE,ID_ATC7,FUENTE
0,1,MORFINA AMP 1% 1 ML (10 MG/1 ML),5,2020-04-10,2020-04-10,ALCALOIDES NATURALES DEL OPIO,N02AA,MORFINA,N02AA01,F_24_04_2020
1,1,DOLQUINE COMP 200 MG,2,2020-04-06,2020-04-10,AMINOQUINOLINAS,P01BA,HIDROXICLOROQUINA,P01BA02,F_24_04_2020
2,1,PARACETAMOL COMP 1 G,3,2020-04-07,2020-04-10,ANILIDAS,N02BE,PARACETAMOL,N02BE01,F_24_04_2020
3,1,KALETRA/ALUVIA COMP (200+50) MG,4,2020-04-07,2020-04-07,ANTIVIRALES PARA EL TRATAMIENTO DE INFECCIONES...,J05AR,RITONAVIR Y LOPINAVIR,J05AE20,F_24_04_2020
4,1,DESINCLOR JABONOSO SOL 4% F/500 ML,2,2020-04-06,2020-04-06,BIGUANIDAS Y AMIDINAS,D08AC,CLORHEXIDINA,D08AC02,F_24_04_2020
...,...,...,...,...,...,...,...,...,...,...
251736,2788,SUERO FISIOLOGICO LAV. 100 ML,1,2020-04-02,2020-04-02,SOLUCIONES ELECTROLÍTICAS,B05XA,CLORURO DE SODIO,B05XA03,F_19_04_2021
251737,2789,AZITROMICINA COMP 500 MG,1,2020-03-24,2020-03-27,MACRÓLIDOS,J01FA,AZITROMICINA,J01FA10,F_19_04_2021
251738,2789,OXIGENO PLANTA,22,2020-03-23,2020-04-03,GASES MEDICINALES,V03AN,OXÍGENO,V03AN01,F_19_04_2021
251739,2789,SUERO CLNA 100 ML I.V.,2,2020-03-23,2020-04-03,SOLUCIONES QUE AFECTAN EL BALANCE ELECTROLÍTICO,B05BB,ELECTROLITOS,B05BB01,F_19_04_2021


In [44]:
#CONVERTIR COLUMNAS EN FECHAS PARA QUE MONGO LAS DETECTE COMO TIPO DATE
# INICIO_TRATAMIENTO         
# FIN_TRATAMIENTO

columnas_fechas = ['INICIO_TRATAMIENTO', 'FIN_TRATAMIENTO']
df_medicacion =  convierteFecha(df_medicacion, columnas_fechas)

In [45]:
df_medicacion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251741 entries, 0 to 251740
Data columns (total 10 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   ID_PACIENTE               251741 non-null  int64         
 1   FARMACO_NOMBRE_COMERCIAL  251741 non-null  object        
 2   DOSIS_MEDIA_DIARIA        251741 non-null  int64         
 3   INICIO_TRATAMIENTO        251741 non-null  datetime64[ns]
 4   FIN_TRATAMIENTO           251741 non-null  datetime64[ns]
 5   ATC5_NOMBRE               251741 non-null  object        
 6   ID_ATC5                   251741 non-null  object        
 7   ATC7_NOMBRE               251736 non-null  object        
 8   ID_ATC7                   251736 non-null  object        
 9   FUENTE                    251741 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(6)
memory usage: 19.2+ MB


In [ ]:
#Insertamos Medicación en MongoDB
mongoInsertMany(df_medicacion,"medicacion")

Datos ingresados correctamente en  Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'hospital_db') --->  medicacion


## 3. Constantes Vitales

In [67]:
url = "Limpios/03_constantes_limpio.csv";
df_constantes =  pd.read_csv(url,delimiter=DetectaSeparador(url),encoding=DetectaEncoding(url))
df_constantes

,ID_PACIENTE,FECHA_CONSTANTE_INGRESO,FC_INGRESO,GLU_INGRESO,SAT_02_INGRESO,TA_MAX_INGRESO,TA_MIN_INGRESO,TEMPERATURA_INGRESO,FUENTE,SAT_02_INGRESO_OBSERVACIONES
0,2316,2020-04-24 00:00:00,62.0,NaN,NaN,NaN,NaN,36.5,F_24_04_2020,NaN
1,2316,2020-04-24 00:01:00,69.0,NaN,NaN,NaN,NaN,37.1,F_24_04_2020,NaN
2,2288,2020-04-24 00:04:00,94.0,NaN,NaN,NaN,NaN,36.8,F_24_04_2020,NaN
3,514,2020-04-24 00:09:00,NaN,NaN,NaN,NaN,NaN,35.4,F_24_04_2020,NaN
4,181,2020-04-24 00:10:00,NaN,NaN,NaN,NaN,NaN,36.4,F_24_04_2020,NaN
...,...,...,...,...,...,...,...,...,...,...
407661,542,2020-05-17 10:27:00,NaN,NaN,98.0,NaN,NaN,NaN,F_19_04_2021,SATO2 GN 1L
407662,605,2020-05-17 11:55:00,NaN,NaN,96.0,NaN,NaN,NaN,F_19_04_2021,NaN
407663,580,2020-11-08 15:46:00,NaN,NaN,35.0,NaN,NaN,NaN,F_19_04_2021,SATO2 GN 4L
407664,4298,2020-11-08 15:50:00,NaN,NaN,97.0,NaN,NaN,NaN,F_19_04_2021,SATO2 GN 1L


In [57]:
#CONVERTIR COLUMNAS EN FECHAS PARA QUE MONGO LAS DETECTE COMO TIPO DATE
# FECHA_CONSTANTE_INGRESO         

columnas_fechas = ['FECHA_CONSTANTE_INGRESO']
df_constantes =  convierteFecha(df_constantes, columnas_fechas)

In [49]:
df_constantes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 407666 entries, 0 to 407665
Data columns (total 10 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   ID_PACIENTE                   407666 non-null  int64         
 1   FECHA_CONSTANTE_INGRESO       407666 non-null  datetime64[ns]
 2   FC_INGRESO                    171968 non-null  float64       
 3   GLU_INGRESO                   0 non-null       float64       
 4   SAT_02_INGRESO                171312 non-null  float64       
 5   TA_MAX_INGRESO                109834 non-null  float64       
 6   TA_MIN_INGRESO                109793 non-null  float64       
 7   TEMPERATURA_INGRESO           186587 non-null  float64       
 8   FUENTE                        407666 non-null  object        
 9   SAT_02_INGRESO_OBSERVACIONES  329016 non-null  object        
dtypes: datetime64[ns](1), float64(6), int64(1), object(2)
memory usage: 31.1+ MB


In [50]:
#Insertamos Medicación en MongoDB
mongoInsertMany(df_constantes,"constantes_vitales")

Datos ingresados correctamente en  Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'hospital_db') --->  constantes_vitales


## 4.Laboratorio

In [71]:
url = "Limpios/04_laboratorio_limpio.csv";
df_laboratorio =  pd.read_csv(url,delimiter=DetectaSeparador(url),encoding=DetectaEncoding(url))
df_laboratorio

,ID_PACIENTE,LAB_NUMERO,LAB_FECHA,LAB_ITEM,RESULTADO_VALOR,RESULTADO_UNIDAD,RESULTADO_VALORES_REFERENCIALES,FUENTE
0,1,I-1,2020-04-08 00:00:00,GOT -- GOT (AST),215.0,U/L,<41,F_24_04_2020
1,1,I-1,2020-04-08 00:00:00,LIN% -- LINFOCITOS %,3.0,%,(20.0 - 50.0),F_24_04_2020
2,1,I-1,2020-04-08 00:00:00,MONO -- MONOCITOS,0.12,X10E3/ÁL,(0.1 - 0.8),F_24_04_2020
3,1,I-2,2020-04-07 00:00:00,BAS -- BASÓFILOS,0.02,X10E3/ÁL,(0.0 - 0.3),F_24_04_2020
4,1,I-2,2020-04-07 00:00:00,BT -- BILIRRUBINA TOTAL ...,0.43,MG/DL,<1.2,F_24_04_2020
...,...,...,...,...,...,...,...,...
1767077,4474,I-1,2020-12-04 00:00:00,G-CORONAV (RT-PCR) -- TIPO DE MUESTRA: EXUDADO,NO SE DETECTA,NaN,NaN,F_19_04_2021
1767078,4474,I-2,2020-12-10 00:00:00,G-CORONAV (RT-PCR) -- TIPO DE MUESTRA: EXUDADO,SE DETECTA,NaN,NaN,F_19_04_2021
1767079,4475,I-1,2020-12-10 00:00:00,G-CORONAV (RT-PCR) -- TIPO DE MUESTRA: EXUDADO,NO SE DETECTA,NaN,NaN,F_19_04_2021
1767080,4478,I-1,2020-12-04 00:00:00,G-CORONAV (RT-PCR) -- TIPO DE MUESTRA: EXUDADO,SE DETECTA,NaN,NaN,F_19_04_2021


In [72]:
#CONVERTIR COLUMNAS EN FECHAS PARA QUE MONGO LAS DETECTE COMO TIPO DATE
# LAB_FECHA         

columnas_fechas = ['LAB_FECHA']
df_laboratorio =  convierteFecha(df_laboratorio, columnas_fechas)

In [ ]:
df_laboratorio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1767082 entries, 0 to 1767081
Data columns (total 8 columns):
 #   Column                           Dtype         
---  ------                           -----         
 0   ID_PACIENTE                      int64         
 1   LAB_NUMERO                       object        
 2   LAB_FECHA                        datetime64[ns]
 3   LAB_ITEM                         object        
 4   RESULTADO_VALOR                  object        
 5   RESULTADO_UNIDAD                 object        
 6   RESULTADO_VALORES_REFERENCIALES  object        
 7   FUENTE                           object        
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 107.9+ MB


In [77]:
#Insertamos Medicación en MongoDB
mongoInsertMany(df_laboratorio,"laboratorio")

Datos ingresados correctamente en  Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'hospital_db') --->  laboratorio


## 5. CIE10 Urgencias

In [75]:
url = "Limpios/05_cie10_urgencia_limpio.csv";
df_cie10_urgencia =  pd.read_csv(url,delimiter=DetectaSeparador(url),encoding=DetectaEncoding(url))
df_cie10_urgencia

,ID_PACIENTE,DIA_PPAL,DIA_02,DIA_03,DIA_04,DIA_05,DIA_06,DIA_07,DIA_08,DIA_09,DIA_10,DIA_11,DIA_12,PROC_01,PROC_02,PROC_03,PROC_04,PROC_05,FUENTE
0,47,J12.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_24_04_2020
1,48,J98.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_24_04_2020
2,49,I50.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_24_04_2020
3,50,J98.4,N39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_24_04_2020
4,51,J18.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_24_04_2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8633,2758,E86.0,E87.1,J12.89,J84.9,R68.89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BW03ZZZ,0BJ0XZZ,NaN,NaN,NaN,F_19_04_2021
8634,2759,J96.90,R68.89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BW03ZZZ,NaN,NaN,NaN,NaN,F_19_04_2021
8635,2760,R06.00,J12.89,B97.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BW03ZZZ,NaN,NaN,NaN,NaN,F_19_04_2021
8636,2761,R68.89,J96.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BW03ZZZ,NaN,NaN,NaN,NaN,F_19_04_2021


In [76]:
df_cie10_urgencia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8638 entries, 0 to 8637
Data columns (total 19 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID_PACIENTE  8638 non-null   int64 
 1   DIA_PPAL     8638 non-null   object
 2   DIA_02       6730 non-null   object
 3   DIA_03       4343 non-null   object
 4   DIA_04       2239 non-null   object
 5   DIA_05       1179 non-null   object
 6   DIA_06       593 non-null    object
 7   DIA_07       319 non-null    object
 8   DIA_08       182 non-null    object
 9   DIA_09       95 non-null     object
 10  DIA_10       63 non-null     object
 11  DIA_11       35 non-null     object
 12  DIA_12       20 non-null     object
 13  PROC_01      4849 non-null   object
 14  PROC_02      1605 non-null   object
 15  PROC_03      154 non-null    object
 16  PROC_04      30 non-null     object
 17  PROC_05      9 non-null      object
 18  FUENTE       8638 non-null   object
dtypes: int64(1), object(18)
mem

In [ ]:
#Insertamos CIE10 Urgencia en MongoDB
mongoInsertMany(df_cie10_urgencia,"cie10_urgencia")

Datos ingresados correctamente en  Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'hospital_db') --->  cie10_urgencia


## 6. CIE10 Ingresos

In [79]:
url = "Limpios/06_cie10_ingreso_limpio.csv";
df_cie10_ingreso =  pd.read_csv(url,delimiter=DetectaSeparador(url),encoding=DetectaEncoding(url))
df_cie10_ingreso

C:\Users\ruben\AppData\Local\Temp\ipykernel_13936\4268050105.py:2: DtypeWarning: Columns (23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cie10_ingreso =  pd.read_csv(url,delimiter=DetectaSeparador(url),encoding=DetectaEncoding(url))


,ID_PACIENTE,DIA_PPAL,DIA_02,DIA_03,DIA_04,DIA_05,DIA_06,DIA_08,DIA_07,DIA_09,...,PROC_12,PROC_13,PROC_14,PROC_15,PROC_16,PROC_17,PROC_18,PROC_19,PROC_20,FUENTE
0,2293,J18.9,Z20.828,R68.89,R09.02,D80.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_24_04_2020
1,2287,J12.89,B97.29,M19.90,F32.9,G25.81,M51.86,Z96.1,Z98.49,Z60.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_24_04_2020
2,2269,J18.9,R68.89,J96.90,I10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_24_04_2020
3,2265,J12.89,B97.29,J96.00,J84.89,Z99.89,Z95.0,NaN,Z79.52,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_24_04_2020
4,2264,J47.0,J12.89,B97.29,J47.1,I10,E11.9,Z87.891,K44.9,Z98.52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_24_04_2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8720,2904,R05,Z20.828,R68.89,K29.50,Z98.52,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_19_04_2021
8721,2912,J98.8,R68.89,Z20.828,R79.89,L50.3,Z86.19,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_19_04_2021
8722,2916,G44.89,R68.89,Z20.828,R78.81,B95.7,M93.88,M47.812,I35.0,F33.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_19_04_2021
8723,2956,J18.9,Z20.828,I48.91,J90,I11.0,I50.9,H81.399,E11.9,H91.90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_19_04_2021


In [80]:
df_cie10_ingreso.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8725 entries, 0 to 8724
Data columns (total 66 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID_PACIENTE  8725 non-null   int64  
 1   DIA_PPAL     8725 non-null   object 
 2   DIA_02       8697 non-null   object 
 3   DIA_03       8163 non-null   object 
 4   DIA_04       7352 non-null   object 
 5   DIA_05       6562 non-null   object 
 6   DIA_06       5726 non-null   object 
 7   DIA_08       4247 non-null   object 
 8   DIA_07       4993 non-null   object 
 9   DIA_09       3658 non-null   object 
 10  DIA_10       3092 non-null   object 
 11  DIA_11       2599 non-null   object 
 12  DIA_12       2168 non-null   object 
 13  DIA_13       1814 non-null   object 
 14  DIA_14       1478 non-null   object 
 15  DIA_15       1210 non-null   object 
 16  DIA_16       992 non-null    object 
 17  DIA_17       788 non-null    object 
 18  DIA_18       628 non-null    object 
 19  DIA_19

In [ ]:
#Insertamos CIE10 Ingresos en MongoDB
mongoInsertMany(df_cie10_ingreso,"cie10_ingreso")

Datos ingresados correctamente en  Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'hospital_db') --->  cie10_ingreso


## 7. CIE10 Códigos Diagnosticos

In [84]:
url = "Limpios/07_cie10_codigos_diagnosticos_limpio.csv";
df_cie10_diagnosticos =  pd.read_csv(url,delimiter=DetectaSeparador(url),encoding=DetectaEncoding(url))
df_cie10_diagnosticos

,ID,CODIGO,NIVEL,DESCRIPCION_CORTA,DESCRIPCION_LARGA
0,1,A00,0,CHOLERA,CHOLERA
1,2,A000,1,"CHOLERA DUE TO VIBRIO CHOLERAE 01, BIOVAR CHOL...","CHOLERA DUE TO VIBRIO CHOLERAE 01, BIOVAR CHOL..."
2,3,A001,1,"CHOLERA DUE TO VIBRIO CHOLERAE 01, BIOVAR ELTOR","CHOLERA DUE TO VIBRIO CHOLERAE 01, BIOVAR ELTOR"
3,4,A009,1,"CHOLERA, UNSPECIFIED","CHOLERA, UNSPECIFIED"
4,5,A01,0,TYPHOID AND PARATYPHOID FEVERS,TYPHOID AND PARATYPHOID FEVERS
...,...,...,...,...,...
95353,95354,Z9981,1,DEPENDENCE ON SUPPLEMENTAL OXYGEN,DEPENDENCE ON SUPPLEMENTAL OXYGEN
95354,95355,Z9989,1,DEPENDENCE ON OTHER ENABLING MACHINES AND DEVICES,DEPENDENCE ON OTHER ENABLING MACHINES AND DEVICES
95355,95356,U07,0,EMERGENCY USE OF U07,EMERGENCY USE OF U07
95356,95357,U070,1,VAPING-RELATED DISORDER,VAPING-RELATED DISORDER


In [85]:
df_cie10_diagnosticos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95358 entries, 0 to 95357
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ID                 95358 non-null  int64 
 1   CODIGO             95358 non-null  object
 2   NIVEL              95358 non-null  int64 
 3   DESCRIPCION_CORTA  95358 non-null  object
 4   DESCRIPCION_LARGA  95358 non-null  object
dtypes: int64(2), object(3)
memory usage: 3.6+ MB


In [ ]:

#Insertamos CIE10 Diagnosticos en MongoDB
mongoInsertMany(df_cie10_diagnosticos,"cie10_diagnostico")

Datos ingresados correctamente en  Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'hospital_db') --->  cie10_diagnostico
